<a href="https://colab.research.google.com/github/ashish25ece/Hackathon/blob/master/Passenger%20Level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

from sklearn.metrics import mean_squared_error

from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor 

In [0]:
from google.colab import files
uploaded = files.upload()


Saving Test.csv to Test.csv
Saving Train.csv to Train.csv


In [0]:
import io
df_train = pd.read_csv(io.BytesIO(uploaded['Train.csv']))
df_test  = pd.read_csv(io.BytesIO(uploaded['Test.csv']))


In [0]:
df_train.isnull().sum()

id_code                         0
current_date                    0
current_time                    0
source_name                     0
destination_name                0
train_name                      0
target                          0
country_code_source             1
longitude_source                1
latitude_source                 1
mean_halt_times_source          1
country_code_destination       33
longitude_destination          33
latitude_destination           33
mean_halt_times_destination    33
current_year                    0
current_week                    0
current_day                     0
is_weekend                      0
dtype: int64

In [0]:
df_train.dropna(inplace=True)

In [0]:
df= df_train.append(df_test)

In [0]:
df.isnull().sum()

country_code_destination         0
country_code_source              0
current_date                     0
current_day                      0
current_time                     0
current_week                     0
current_year                     0
destination_name                 0
id_code                          0
is_weekend                       0
latitude_destination             0
latitude_source                  0
longitude_destination            0
longitude_source                 0
mean_halt_times_destination      0
mean_halt_times_source           0
source_name                      0
target                         964
train_name                       0
dtype: int64

In [0]:
df.head()

,country_code_destination,country_code_source,current_date,current_day,current_time,current_week,current_year,destination_name,id_code,is_weekend,latitude_destination,latitude_source,longitude_destination,longitude_source,mean_halt_times_destination,mean_halt_times_source,source_name,target,train_name
31,whber,whber,2016-07-27,Wednesday,11:41:46 PM,30,2016,station$214,mgtoldpcbopaoba,False,51.017648,50.859663,4.482785,4.360846,306.523121,640.265896,station$130,high,ICWVYA
32,whber,whber,2016-07-28,Thursday,12:04:25 AM,30,2016,station$248,hbvekntxnxearzq,False,50.967057,50.967057,5.497685,5.497685,39.476879,39.476879,station$248,low,ICZVXY
34,whber,whber,2016-07-28,Thursday,12:18:09 AM,30,2016,station$147,rfwvjolvzoekxsq,False,50.845658,50.967057,4.356801,5.497685,634.164740,39.476879,station$248,low,ICZVXY
35,whber,whber,2016-07-28,Thursday,12:38:28 AM,30,2016,station$130,jcbfhwsqzpcifpe,False,50.859663,50.896456,4.360846,4.482076,640.265896,171.020231,station$177,high,ICXUYS
36,whber,whber,2016-07-28,Thursday,08:50:27 AM,30,2016,station$147,qfnccvisvjzlkxg,False,50.845658,51.035896,4.356801,3.710675,634.164740,309.014451,station$544,medium,ICZVAT


In [0]:
a=df['current_time'].str.split(':')
b=a.str[2].str.split()
df['Hours']=a.str[0]
df['Minutes']=a.str[1]
df['Seconds']=b.str[0]
df['Merridean']=b.str[1]

In [0]:
df.columns

Index(['country_code_destination', 'country_code_source', 'current_day',
       'destination_name', 'is_weekend', 'latitude_destination',
       'latitude_source', 'longitude_destination', 'longitude_source',
       'mean_halt_times_destination', 'mean_halt_times_source', 'source_name',
       'target', 'train_name', 'Hours', 'Minutes', 'Seconds', 'Merridean'],
      dtype='object')

In [0]:
df=df.drop(labels=['current_date', 'current_time', 'current_week', 'current_year','id_code'],axis=1)

In [0]:
df.country_code_destination.unique()

array(['whber', 'aqfre', 'qwnll', 'zxdec', 'logbp', 'wsluu'], dtype=object)

In [0]:
preprocess = ColumnTransformer(
    [('country_code_destination', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['country_code_destination']),
     ('country_code_source', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['country_code_source']),
     ('current_day', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['current_day']),
     ('destination_name', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['destination_name']),
     ('is_weekend', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['is_weekend']),
     ('source_name', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['source_name']),
     ('train_name', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['train_name']),
     ('Merridean', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['Merridean'])])
#     ('target', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['target'])
#     ('Date_of_Publishing_Month', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['Date_of_Publishing_Month']),
#     ('Date_of_Publishing_Year', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['Date_of_Publishing_Year']),
#     ('Printed_category', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['Printed']),
#     ('Title_countvec', CountVectorizer(min_df=10), 'Title'),
#     ('synopsis_tfidf', TfidfVectorizer(max_features = 2000, stop_words = 'english', ngram_range=(1,3)), 'Synopsis')],
#    remainder='passthrough')


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2214 entries, 31 to 963
Data columns (total 18 columns):
country_code_destination       2214 non-null object
country_code_source            2214 non-null object
current_day                    2214 non-null object
destination_name               2214 non-null object
is_weekend                     2214 non-null bool
latitude_destination           2214 non-null float64
latitude_source                2214 non-null float64
longitude_destination          2214 non-null float64
longitude_source               2214 non-null float64
mean_halt_times_destination    2214 non-null float64
mean_halt_times_source         2214 non-null float64
source_name                    2214 non-null object
target                         1250 non-null object
train_name                     2214 non-null object
Hours                          2214 non-null object
Minutes                        2214 non-null object
Seconds                        2214 non-null object
Merri

In [0]:
df_train.shape

(1250, 19)

In [0]:
df_train = df[0:1250]
df_test = df[1250:]
df_test = df_test.drop(['target'], axis =1)

X = df_train.drop(['target'],axis = 1)
y = df_train['target']



df_X=pd.DataFrame(preprocess.fit_transform(X).todense())

df_X.shape


(1250, 980)

In [0]:
# Segregate the data set into train and test set

from sklearn.model_selection import train_test_split, KFold, cross_val_score

train_X,test_X,train_y,test_y = train_test_split(df_X,y,test_size=0.2,random_state=0)



In [0]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [0]:
models=[]
results=[]
names=[]

models.append(('LR',LogisticRegression()))
models.append(('KNN',KNeighborsClassifier()))
models.append(('DT',DecisionTreeClassifier()))
models.append(('NB',GaussianNB()))
models.append(('RFC',RandomForestClassifier()))
models.append(('SVM',SVC()))

models

[('LR',
  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                     intercept_scaling=1, l1_ratio=None, max_iter=100,
                     multi_class='warn', n_jobs=None, penalty='l2',
                     random_state=None, solver='warn', tol=0.0001, verbose=0,
                     warm_start=False)),
 ('KNN',
  KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                       metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                       weights='uniform')),
 ('DT',
  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                         max_features=None, max_leaf_nodes=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=1, min_samples_split=2,
                         min_weight_fraction_leaf=0.0, presort=False,
                         random_state=None, splitter='best')),
 ('NB', GaussianNB(prior

In [0]:
for name,model in models:
    k_folds = StratifiedKFold(n_splits=10, random_state=42)
    cv_score= cross_val_score(model,train_X,y=train_y,cv=k_folds,scoring='accuracy')
    results.append(cv_score)
    names.append(name)
    print("%s: %f (%f)" % (name, cv_score.mean()*100, cv_score.std()))

LR: 52.508456 (0.030815)
KNN: 48.393705 (0.034213)
DT: 49.987988 (0.050237)
NB: 48.992857 (0.032669)
RFC: 53.093422 (0.025669)
SVM: 41.500840 (0.002774)


In [0]:
#Since Logistic Regression is the best fitted model

rfc = RandomForestClassifier()
rfc.fit(train_X,train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
rfc_predicted = rfc.predict(test_X)
rfc_predicted

array(['medium', 'low', 'high', 'high', 'low', 'medium', 'medium',
       'medium', 'medium', 'low', 'low', 'high', 'low', 'high', 'high',
       'low', 'low', 'low', 'high', 'high', 'medium', 'low', 'low',
       'high', 'high', 'low', 'high', 'low', 'low', 'medium', 'low',
       'high', 'medium', 'low', 'high', 'low', 'low', 'low', 'low',
       'high', 'high', 'low', 'low', 'low', 'low', 'low', 'low', 'medium',
       'low', 'low', 'low', 'low', 'high', 'low', 'low', 'low', 'high',
       'medium', 'high', 'medium', 'low', 'high', 'low', 'low', 'medium',
       'low', 'low', 'low', 'low', 'low', 'low', 'high', 'medium', 'low',
       'low', 'high', 'high', 'low', 'low', 'medium', 'low', 'high',
       'low', 'low', 'low', 'low', 'high', 'medium', 'high', 'low', 'low',
       'low', 'low', 'low', 'high', 'high', 'low', 'low', 'high', 'high',
       'low', 'medium', 'low', 'high', 'low', 'low', 'low', 'low',
       'medium', 'high', 'low', 'low', 'medium', 'low', 'high', 'medium',
  

In [0]:
#rfc_score=f1_score(test_y,rfc_predicted)

print ('RFC Model :')
#print ('F1 score : %.2f' %(f1_score*100))
print ('Classfication report : ')
print (classification_report(test_y,rfc_predicted))
print ('confusion_matrix : ')
print (confusion_matrix(test_y,rfc_predicted))

RFC Model :
Classfication report : 
              precision    recall  f1-score   support

        high       0.46      0.49      0.47        68
         low       0.62      0.72      0.67       116
      medium       0.47      0.30      0.37        66

    accuracy                           0.55       250
   macro avg       0.52      0.50      0.50       250
weighted avg       0.54      0.55      0.54       250

confusion_matrix : 
[[33 25 10]
 [19 84 13]
 [20 26 20]]


In [0]:
df_test = df[1250:]
df_test = df_test.drop(['target'], axis =1)

X = df_test

df_X=pd.DataFrame(preprocess.fit_transform(X).todense())

df_X.shape


(964, 860)

In [0]:
rfc_predicted_test = rfc.predict(df_X)


In [0]:
test_X.shape

(250, 980)